# Using ferret to explain instances from *Thermostat*

This notebook showcases the integration between **ferret** and **[thermostat](https://github.com/DFKI-NLP/thermostat)**, a large collection of pre-computed NLP model explanations for classification tasks.

Specifically, you will see how to:

- load a thermostat dataset;
- visualize the explanations of thermostat dataset of multiple explainers;
- **evaluate** thermostat explanations over the faithfulness metrics.

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from ferret.datasets.datamanagers_thermostat import ThermostatDataset
from ferret import Benchmark
import numpy as np
import torch

## Load Thermostat explanations

*Thermostat* is a collection of pre-computed explanations of NLP models. Explanations are pre-computed for a wide range of HuggingFace datasets and transformers-based models.

ferret easily integrates thermostat.
A thermostat dataset is identified by the three basic information: dataset, model, and explainer.
For loading it, ferret adopts the same input configuration. 

For example, we can load the explanations of lime for the bert model on the ag news dataset by specifying the string "ag_news-bert-lime": `thermostat_data_exp = ThermostatDataset("ag_news-bert-lime")`

### Choosing the dataset and precomputed scores

In the example, we load the explanations of the *lime*, *occlusion* and *layer integrated gradients* (lig) explainers for the bert prediction on the ag news dataset. 

In [3]:
thermostat_data = ThermostatDataset("ag_news-bert", name_explainers=["lime", "occlusion", "lig"])

Loading Thermostat configuration: ag_news-bert-lime


The repository for dataset contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home/giuseppe/miniconda3/envs/ferret/lib/python3.10/site-packages/thermostat/dataset.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
Generating test split: 7600 examples [00:10, 759.04 examples/s]


Loading Thermostat configuration: ag_news-bert-occlusion


Generating test split: 7600 examples [00:10, 712.98 examples/s]


Loading Thermostat configuration: ag_news-bert-lig


Generating test split: 7600 examples [00:10, 751.97 examples/s]
tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 48.0/48.0 [00:00<00:00, 88.8kB/s]
config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 706/706 [00:00<00:00, 1.45MB/s]
vocab.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 1.11MB/s]
special_tokens_map.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:00<00:00, 250kB/s]


### Loading model and tokenizer

Thermostat specifies which model and tokenizer were used to generate the attribution scores. Let's retrieve and use them with our `Benchmark` class. 

In [8]:
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device {device}")
model = AutoModelForSequenceClassification.from_pretrained(thermostat_data.model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(thermostat_data.tokenizer_name)

Using device cuda:0


In [10]:
bench = Benchmark(model, tokenizer, task_name="text-classification")

## Explanations of a single instance and its evaluation

We can easily get the explanations for an instance and visualize it.

Note that we can get explanations by its index (`explanations = thermostat_data[i]["explanations"]`) or directly via the `get_explanations` function. We normalize explanations to ease the comparison.

In [13]:
idx = 0
explanations = thermostat_data.get_explanations(idx=idx, normalize_scores=True)
table = bench.show_table(explanations)
table

,fears,for,t,n,pension,after_0,talks_0,unions,representing,workers,at,turner,new,##all,say,they,are,'_0,disappointed,'_1,after_1,talks_1,with,stricken,parent,firm,federal,mo,##gul,.
lime,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00
occlusion,-0.42,-0.15,0.16,0.59,0.42,0.22,-0.11,-0.10,-0.03,-0.06,0.00,0.02,0.02,-0.06,-0.20,-0.18,-0.03,-0.02,0.03,-0.11,-0.03,-0.03,0.04,0.02,-0.03,-0.08,-0.10,-0.19,0.07,0.11
lig,0.87,0.24,-0.96,0.43,1.95,0.38,0.49,1.08,0.17,0.32,0.05,0.15,0.05,-0.13,-0.09,-0.09,-0.58,-0.17,0.53,-0.32,-0.49,0.44,-0.05,0.31,0.85,1.30,0.47,-0.90,0.36,-0.27


Evaluate the pre-computed explanations with all the supported evaluators is straightforward. 
The `target` parameter matches the one used by Thermostat to generate the the explanations of the instance!

In [14]:
target = thermostat_data.get_target_explanations(idx)
explanation_evaluations = bench.evaluate_explanations(explanations, target=target)

In [15]:
bench.show_evaluation_table(explanation_evaluations)

,aopc_compr,aopc_suff,taucorr_loo
Explainer,,,
lime,0.19,0.15,-0.01
occlusion,0.13,-0.00,0.02
lig,0.24,-0.00,-0.06
